# Урок 3

В данном уроке мы попробуем запустить наше DTW уже на реальном звуке. Для начала это будут простые слова "да" и "нет". Имеется небольшая база с файлами различных вариаций произнесения этих слов. Часть из них (эталоны) будут использованны для построения графа. Для остальных же (записей) будет поочередно запущен DTW алгорим для определения ближайшего к ним файла из эталонов.

MFCC признаки мы уже посчитали и сохранили в формате ark в файлах etalons_mfcc.txtftr и records_mfcc.txtftr соответственно.

Ранее, наш граф был способен идти только вровень, либо сжимать запись (оставаться в том же узле графа) относительно кадров эталона. Но необходимо еще уметь и растягивать запись. Для этого нужно ввести дополнительные переходы через один и два состояния для узлов графа.

<br>
<img src="graph.png">
<br>

<b>Задание 1:</b>
Добавить для узлов графа дополнительные переходы через один и два состояния (нулевой узел должен остаться прежним).

In [1]:
import numpy as np
import FtrFile
from scipy.spatial import distance

In [2]:
class State:
    def __init__(self, ftr, idx):
        self.ftr = ftr           # вектор признаков узла 
        self.isFinal = False     # является ли этот узел финальнвм в слове
        self.word = None         # слово эталона (назначается только для финального узла)       
        self.nextStates = []     # список следующих узлов
        self.idx = idx           # индекс узла
        self.bestToken = None    # лучший токен (по минимуму дистанции) в узле
        self.currentWord = None  # текущее слово эталона

In [3]:
def load_graph(rxfilename, num_next=3):
    assert num_next >= 1
    startState = State(None, 0)
    graph = [startState, ]
    stateIdx = 1
    for word, features in FtrFile.FtrDirectoryReader(rxfilename):
        prevStates = [startState, ]
        for frame in range(features.nSamples):
            state = State(features.readvec(), stateIdx)
            state.currentWord = word           # слово эталона теперь будет храниться в каждом узле
            state.nextStates.append(state)  
            for prevState in prevStates:
                prevState.nextStates.append(state) 
            if prevStates[0] == startState: # only root
                prevStates = [state,]
            else: 
                prevStates.append(state)
                prevStates = prevStates[-num_next:]
            graph.append(state)
            stateIdx += 1
            
        if state:
            state.word = word
            state.isFinal = True
    return graph

Следующий блок кода проверяет ваш граф на некоторые ключевые параметры и записывает в удобном для чтения виде в файл graph.txt. Сравните его с заведомо правильным графом, сохраненным в graph_reference.txt:

In [4]:
def check_graph(graph):
    assert len(graph) > 0, "graph is empty."
    assert graph[0].ftr is None \
        and graph[0].word is None \
        and not graph[0].isFinal, "broken start state in graph."
    idx = 0
    for state in graph:
        assert state.idx == idx
        idx += 1
        assert (state.isFinal and state.word is not None) \
            or (not state.isFinal and state.word is None)


def print_graph(graph):
    with open('graph.txt', 'w') as fn:
        np.set_printoptions(formatter={'float': '{: 0.1f}'.format})
        for state in graph:
            nextStatesIdxs = [s.idx for s in state.nextStates]
            fn.write("State: idx={} word={} isFinal={} nextStatesIdxs={} ftr={} \n".format(
                state.idx, state.word, state.isFinal, nextStatesIdxs, state.ftr))
    print("*** SEE graph.txt ***")
    print("*** END DEBUG. GRAPH ***")

    
etalons = "ark,t:etalons_mfcc.txtftr"
graph = load_graph(etalons)
check_graph(graph)
# Сохранить граф в читабельном виде в файл graph.txt:
print_graph(graph)

*** SEE graph.txt ***
*** END DEBUG. GRAPH ***


Реализованный в прошлом уроке TPA в данном случае будет перебирать все возможные варианты разметки, что приведет к значительному увеличению времени работы нашего DTW. Для решения этой проблемы мы будем отбрасывать "ненужные" токены еще на этапе прохождения по графу. Этим занимаются, так называемые, beam и state prunings.

### state pruning:
В классе State нужно добавить атрибут best_token – ссылку на лучший токен, заканчивающийся в данном стейте на данном кадре записи. После порождения всех токенов за текущий кадр записи, пройдемся по каждому из полученных nextTokens, затем впишем текущий токен в State.best_token (здесь State – это узел, на котором закончился токен), убив предыдущий лучший токен, либо убьем сам токен, если он хуже лучшего на этом узле. За жизнеспособность токена отвечает его атрибут is_alive: True или False соответственно.

После этого необходимо очистить поле best_token у всех узлов графа.

### beam pruning:
Идея состоит в том, чтобы на каждом кадре записи находить плохие токены и откидывать их (token.is_alive = False). 
Плохие – это,очевидно, накопившие слишком большое отклонение от стейтов,по которым они идут. Слишком большое отклонение – это непонятно какое (может токен плохой, может слово слишком длинное, может звук очень плохой – не разобрать).

Поэтому плохость токена считают относительно лучшего токена. Заведем переменную thr_common (обычно её называют beam – ширина луча поиска; у нас это common threshold – “общий порог” – по историческим причинам). И если token.dist > best_token.dist + thr_common, то token плохой и мы его отбросим.

Выкидывая какой-то токен из-за его отклонеиня, мы рискуем тем, что через сколько-то кадров все потомки выживших токенов могут оказаться очень плохими, а только потомки отброшенного токена оказались бы чудо как хороши. То есть, вводя thr_common, мы вводим ошибку.
Поэтому thr_common нужно подобрать так, чтобы скорость сильно выросла, а ошибка выросла незначительно.

<br>
Введение этих методов может привести к тому, что у нас просто не окажется в конце выживших токенов в финальных узлах графа. Для того, чтобы иметь возможность выдавать результат в этом случае, мы введем дополнительный атрибут currentWord у класса State. Теперь в любом узле каждой ветви будет храниться слово соответствующего эталона для этой ветви. 

Тогда в конце работы DTW, если у нас не будет живых финальных токенов, то мы просто выберем лучший из оставшихся и по полю state.currentWord определим слово эталона.

<b>Задание 2:</b> 
- Написать функцию findBest для поиска токена с минимальной дистанцией.
- Реализовать функции для state и beam pruning (здесь нам и может пригодиться функция findBest).
- Разобраться с вычислением WER.

In [5]:
class Token:
    def __init__(self, state, dist=0.0, sentence=""):
        self.state = state
        self.dist = dist
        self.sentence = sentence
        self.alive = True
    

def findBest(Tokens):
    #---------------------------------------TODO---------------------------------  
    bestToken = Tokens[np.argmin([token.dist for token in Tokens])]
    #-----------------------------------------------------------------------------
    return bestToken


def beamPruning(nextTokens, thr_common=70):
#     thr_common = 70 # можно менять
    #--------------------------------TODO--------------------------------------
    # 1. Ищем лучший токен из nextTokens с помощью findBest
    # 2. Присваиваем token.aliv значение False, если дистанция этого токена больше, чем
    #    длина лучшего токена + thr_common
    
    bestToken = findBest(nextTokens)
    bestDist = bestToken.dist
    for token in nextTokens:
        if token.alive and token.dist > bestDist + thr_common:
            token.alive = False
    #--------------------------------------------------------------------------
    return [token for token in nextTokens if token.alive]

In [6]:
def statePruning(nextTokens):
    for token in nextTokens:
        #--------------------------TODO---------------------------------
        if token.state.bestToken is None:
            token.state.bestToken = token
        elif token.state.bestToken.dist > token.dist:
            token.state.bestToken.alive = False
            token.state.bestToken = token
        else: # token.state.bestToken.dist < token.dist
            token.alive = False
        #---------------------------------------------------------------
        
    # Сбросить bestToken на None для всеx узлов графа
    for token in nextTokens:
        token.state.bestToken = None
    #-------------------------------------------------------------------
    return [token for token in nextTokens if token.alive]

In [7]:
def distance(X, Y):
#     result = float(np.sqrt(sum(pow(X - Y, 2))))
    result = np.linalg.norm(X - Y)
#     result = 1 - np.sum(X * Y) / (np.linalg.norm(X) * np.linalg.norm(Y))
#     result = np.sum(np.abs(X - Y))
    return result

In [8]:
def partition(array, l, r):
    elem = array[(l + r) // 2]
    left_arr = [array[i] for i in range(l, r + 1) if array[i] < elem]
    mid_arr = [array[i] for i in range(l, r + 1) if array[i] == elem]
    right_arr = [array[i] for i in range(l, r + 1) if array[i] > elem]
    array[l: r+1] = left_arr + mid_arr + right_arr
    return l + len(left_arr)

In [9]:
def find_k_statistics(array, k):
#     assert len(array) >= k
    k -= 1
    left = 0
    right = len(array) - 1
    while(True):
        mid = partition(array, left, right)
        if mid == k:
            return array[mid]
        elif k < mid:
            right = mid
        else: # k > mid
            left = mid + 1

In [10]:
# 0.0 1.0 2.0, step=1.0, n_buckets = 1
# 2.0 // 1.0

In [11]:
def find_k_statistics_bucket(array, k):
    assert 1 <= k <= len(array)
    current_array = array
    n_buckets = 10
    while True:
        min_arg = min(current_array)
        max_arg = max(current_array)
        if min_arg == max_arg:
            return min_arg
        step = (max_arg - min_arg) / n_buckets
        buckets = [[] for _ in range(n_buckets)]
        for elem in current_array:
            i = int((elem - min_arg) // step) if elem < max_arg else n_buckets - 1
            buckets[i].append(elem)
        i = 0
        while len(buckets[i]) < k:
            k -= len(buckets[i])
            i += 1
        current_array = buckets[i]

In [12]:
# arr = np.random.randint(355, size=(10,)).tolist()
# print(arr)

In [13]:
def filter_best_k(nextTokens, k=100):
#     assert len(nextTokens) > k
    distances = [token.dist for token in nextTokens]
    k_elem = find_k_statistics_bucket(distances, k)
    
    filteredTokens = [token for token in nextTokens if token.dist < k_elem]
    addTokens = [token for token in nextTokens if token.dist == k_elem]
    result = filteredTokens + addTokens[: k - len(filteredTokens)]
    return result

In [53]:
def recognize(features, graph, rec_results, thr_common=70, best_k=None):

    print("-" * 23)
    startTime = time.time()
    startState = graph[0]
    activeTokens = [Token(startState), ]
    nextTokens = []

    for frame in range(features.nSamples):
        ftrCurrentFrameRecord = features.readvec()
        bestNextDist = np.inf
        for token in filter(lambda token: token.alive, activeTokens):
            for i, transitionState in enumerate(token.state.nextStates):
                newDist = token.dist + distance(ftrCurrentFrameRecord, transitionState.ftr)
                if bestNextDist + thr_common >= newDist:
                    if newDist < bestNextDist:
                        bestNextDist = newDist
                        
                    if transitionState.bestToken is None:
                        newToken = Token(transitionState, newDist, token.sentence)
                        nextTokens.append(newToken)
                        transitionState.bestToken = newToken
                    elif transitionState.bestToken.dist > newDist:
                        newToken = Token(transitionState, newDist, token.sentence)
                        nextTokens.append(newToken)
                        transitionState.bestToken.alive = False
                        transitionState.bestToken = newToken
                    else:
                        pass
        for token in nextTokens:
            token.state.bestToken = None           
                    
        # state and beam prunings:
#         nextTokens = statePruning(nextTokens)         
#         nextTokens = beamPruning(nextTokens, thr_common=thr_common) 
        nextTokens = [token for token in nextTokens if  token.alive and token.dist <= bestNextDist + thr_common]
        if best_k is not None and len(nextTokens) > best_k:
            nextTokens = filter_best_k(nextTokens, best_k)
        
        activeTokens = nextTokens
        nextTokens = []                                    
        
    # поиск финальных токенов:
    finalTokens = []
    for token in activeTokens:
        if token.state.isFinal and token.alive:
            finalTokens.append(token)

    # если нет финальных, то берем лучший из выживших:
    if len(finalTokens) != 0:
        winToken = findBest(finalTokens)
    else:
        winToken = findBest(activeTokens)
        winToken.state.word = winToken.state.currentWord

    # вывод результата DTW
    print("result: {} ==> {}".format(filename, winToken.state.word))
    endTime = time.time()
    print("time: {} sec".format(round(endTime-startTime, 2)))

    # совпадает ли запись с полученным эталоном:  
    record_word = filename.split('_')[0]
    etalon_word = winToken.state.word.split('_')[0]
    rec_results.append(etalon_word == record_word)

    return frame

Теперь запустим нашу программу.

In [15]:
import time

etalons = "ark,t:etalons_mfcc.txtftr"
records = "ark,t:records_mfcc.txtftr"

rec_results = []  # переменная для подсчета точности распознавания

s_time = time.time()
numbFrame = 0     # счетчик общего количества кадров для расчета RTF

graph = load_graph(etalons, num_next=3)

for filename, features in FtrFile.FtrDirectoryReader(records):
    frame = recognize(features, graph, rec_results, thr_common=70)
    numbFrame += frame

print("-" * 23)
print("WER is: {}".format(round(1 - sum(rec_results)/len(rec_results), 3)))
e_time = time.time()
time = e_time-s_time
minut = int(time/60)
second = int(time-minut*60)
print("Total time: {} min {} sec".format(minut, second))
rtf = round(time/(numbFrame*0.01), 2)
print("RTF is: {}".format(rtf))

-----------------------
result: da_06 ==> da_02
time: 2.3 sec
-----------------------
result: da_07 ==> da_02
time: 4.74 sec
-----------------------
result: da_08 ==> da_04
time: 3.44 sec
-----------------------
result: da_09 ==> da_02
time: 3.44 sec
-----------------------


KeyboardInterrupt: 

<b>Задание 3:</b> Подбирите значение порога thr_common и количество дополнительных переходов для узлов так, чтобы получить минимально возможно значение WER для данной базы.

In [41]:
import multiprocessing

In [54]:
import time

etalons = "ark,t:etalons_mfcc.txtftr"
records = "ark,t:records_mfcc.txtftr"

rec_results = []  # переменная для подсчета точности распознавания

s_time = time.time()
numbFrame = 0     # счетчик общего количества кадров для расчета RTF

graph = load_graph(etalons, num_next=3)

for filename, features in FtrFile.FtrDirectoryReader(records): 
    frame = recognize(features, graph, rec_results, thr_common=75) #best_k=300
    numbFrame += frame

print("-" * 23)
print("WER is: {}".format(round(1 - sum(rec_results)/len(rec_results), 3)))
e_time = time.time()
time = e_time-s_time
minut = int(time/60)
second = int(time-minut*60)
print("Total time: {} min {} sec".format(minut, second))
rtf = round(time/(numbFrame*0.01), 2)
print("RTF is: {}".format(rtf))

-----------------------
result: da_06 ==> da_02
time: 2.75 sec
-----------------------
result: da_07 ==> da_02
time: 4.86 sec
-----------------------
result: da_08 ==> da_04
time: 3.29 sec
-----------------------
result: da_09 ==> da_02
time: 3.27 sec
-----------------------
result: da_10 ==> da_04
time: 4.24 sec
-----------------------
result: da_11 ==> da_02
time: 3.9 sec
-----------------------
result: da_12 ==> da_05
time: 2.89 sec
-----------------------
result: da_13 ==> da_02
time: 3.01 sec
-----------------------
result: da_14 ==> da_02
time: 2.73 sec
-----------------------
result: da_15 ==> da_02
time: 4.94 sec
-----------------------
result: da_16 ==> da_02
time: 2.28 sec
-----------------------
result: da_17 ==> da_05
time: 3.82 sec
-----------------------
result: da_18 ==> da_05
time: 2.14 sec
-----------------------
result: da_19 ==> da_05
time: 3.9 sec
-----------------------
result: net_06 ==> net_02
time: 3.13 sec
-----------------------
result: net_07 ==> net_04
time:

In [ ]:
# без best_k, state_pruning в процессе Total time: 1 min 56 sec
# без best_k, без state_pruning в процессе Total time: 1 min 54 sec
# all prunings :) Total time: 1 min 48 sec

In [ ]:
# num_next=3, thr_common=150, best_k=1000, euclidean: WER is: 0.0 Total time: 4 min 25 sec
# num_next=3, thr_common=150, best_k=200, euclidean: WER is: 0.033 Total time: 1 min 20 sec
# num_next=3, thr_common=150, best_k=500, euclidean: WER is: 0.0 Total time: 2 min 48 sec
# num_next=3, thr_common=120, best_k=500, euclidean: WER is: 0.0 Total time: 2 min 39 sec
# num_next=3, thr_common=100, best_k=500, euclidean: WER is: 0.0 Total time: 2 min 20 sec
# num_next=3, thr_common=100, best_k=350, euclidean: WER is: 0.0 Total time: 1 min 49 sec
# num_next=3, thr_common=90, best_k=350, euclidean: WER is: 0.0 Total time: 1 min 44 sec
# num_next=3, thr_common=90, best_k=300, euclidean: WER is: 0.0 Total time: 1 min 35 sec
# num_next=3, thr_common=80, best_k=300, euclidean: WER is: 0.0 Total time: 1 min 28 sec
# num_next=3, thr_common=75, best_k=300, euclidean: WER is: 0.0 Total time: 1 min 26 sec

# num_next=3, thr_common=80, best_k=250, euclidean: WER is: 0.033 Total time: 1 min 19 sec

In [76]:
# experiments
# 1 / 150 => 0.033 wer
# 1 / 200 => 0.067 wer
# 1 / 100 => 0.033 wer
# 1 / 80 => 0.0 wer
# 2 / 200 => высокий wer
# 2 / 50 => высокий wer
# 2 / 180 => высокий wer
# 3 / 100 => высокий wer?
# 5 / 30 => 0.233 wer
# 5 / 70 => высокий wer

In [ ]:
# k = 200 good choice?

In [ ]:
# как сэкономить на создании классов?